In [1]:
import os
import sys

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from definitions import ROOT_DIR

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, GaussianNoise

from src.data.make_dataset import CRCHistoPhenotypes_Patch

Using TensorFlow backend.


In [2]:
dataset = CRCHistoPhenotypes_Patch(patch_size=(27, 27), pad_mode="symmetric")
X, y = dataset.load_patches()
labelencoder = LabelEncoder()
labelencoder.fit(y)
y = labelencoder.transform(y)

In [3]:
batch_size = 32
num_classes = 4
epochs = 200
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.33, random_state=42)
X_train = np.concatenate(X_train).reshape((len(X_train),)+X[0].shape)
X_test = np.concatenate(X_test).reshape((len(X_test),)+X[0].shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
model = Sequential()
model.add(GaussianNoise(stddev=0.1, input_shape=(27, 27, 3)))
model.add(Convolution2D(filters=32, kernel_size=(7, 7), activation='relu', input_shape=(27, 27, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Convolution2D(filters=48, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Convolution2D(filters=64, kernel_size=(3, 3), activation='relu'))
# model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128, activation='tanh'))
# model.add(Dropout(0.25))
model.add(Dense(64, activation='tanh'))
# model.add(Dropout(0.25))
model.add(Dense(4, activation='softmax'))


In [6]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=["accuracy"])

In [7]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.0,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(X_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(X_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(X_test, y_test),
                        workers=4)


Using real-time data augmentation.
Epoch 1/200
469/469 [==============================] - 7s - loss: 1.1109 - acc: 0.5290 - val_loss: 0.8920 - val_acc: 0.6622
Epoch 2/200
469/469 [==============================] - 3s - loss: 0.9125 - acc: 0.6511 - val_loss: 0.8574 - val_acc: 0.6706
Epoch 3/200
469/469 [==============================] - 3s - loss: 0.8783 - acc: 0.6600 - val_loss: 0.8342 - val_acc: 0.6787
Epoch 4/200
469/469 [==============================] - 3s - loss: 0.8561 - acc: 0.6709 - val_loss: 0.8816 - val_acc: 0.6602
Epoch 5/200
469/469 [==============================] - 3s - loss: 0.8365 - acc: 0.6757 - val_loss: 0.8081 - val_acc: 0.6887
Epoch 6/200
469/469 [==============================] - 3s - loss: 0.8226 - acc: 0.6850 - val_loss: 0.7628 - val_acc: 0.7034
Epoch 7/200
469/469 [==============================] - 3s - loss: 0.8103 - acc: 0.6879 - val_loss: 0.7945 - val_acc: 0.6973
Epoch 8/200
469/469 [==============================] - 3s - loss: 0.7983 - acc: 0.6912 - val_loss

469/469 [==============================] - 3s - loss: 0.7034 - acc: 0.7370 - val_loss: 0.7095 - val_acc: 0.7365
Epoch 67/200
469/469 [==============================] - 3s - loss: 0.7073 - acc: 0.7343 - val_loss: 0.7014 - val_acc: 0.7361
Epoch 68/200
469/469 [==============================] - 3s - loss: 0.7006 - acc: 0.7382 - val_loss: 0.7054 - val_acc: 0.7350
Epoch 69/200
469/469 [==============================] - 3s - loss: 0.7040 - acc: 0.7346 - val_loss: 0.6801 - val_acc: 0.7421
Epoch 70/200
469/469 [==============================] - 3s - loss: 0.7037 - acc: 0.7341 - val_loss: 0.6845 - val_acc: 0.7425
Epoch 71/200
469/469 [==============================] - 3s - loss: 0.7021 - acc: 0.7381 - val_loss: 0.6778 - val_acc: 0.7412
Epoch 72/200
469/469 [==============================] - 3s - loss: 0.7020 - acc: 0.7392 - val_loss: 0.7537 - val_acc: 0.7192
Epoch 73/200
469/469 [==============================] - 3s - loss: 0.7071 - acc: 0.7361 - val_loss: 0.6926 - val_acc: 0.7458
Epoch 74/200


469/469 [==============================] - 3s - loss: 0.7994 - acc: 0.7067 - val_loss: 0.8218 - val_acc: 0.6877
Epoch 132/200
469/469 [==============================] - 3s - loss: 0.7649 - acc: 0.7170 - val_loss: 0.7278 - val_acc: 0.7286
Epoch 133/200
469/469 [==============================] - 3s - loss: 0.7707 - acc: 0.7158 - val_loss: 0.7837 - val_acc: 0.7111
Epoch 134/200
469/469 [==============================] - 3s - loss: 0.7757 - acc: 0.7193 - val_loss: 0.7317 - val_acc: 0.7347
Epoch 135/200
469/469 [==============================] - 3s - loss: 0.8038 - acc: 0.7062 - val_loss: 0.8682 - val_acc: 0.6920
Epoch 136/200
469/469 [==============================] - 3s - loss: 0.7759 - acc: 0.7136 - val_loss: 0.8621 - val_acc: 0.6737
Epoch 137/200
469/469 [==============================] - 3s - loss: 0.7679 - acc: 0.7159 - val_loss: 0.7591 - val_acc: 0.7177
Epoch 138/200
469/469 [==============================] - 3s - loss: 0.7965 - acc: 0.7038 - val_loss: 0.7689 - val_acc: 0.7124
Epoch 

469/469 [==============================] - 3s - loss: 0.8326 - acc: 0.6880 - val_loss: 0.8007 - val_acc: 0.7015
Epoch 197/200
469/469 [==============================] - 3s - loss: 0.9247 - acc: 0.6514 - val_loss: 0.9106 - val_acc: 0.6619
Epoch 198/200
469/469 [==============================] - 3s - loss: 0.8494 - acc: 0.6816 - val_loss: 0.7684 - val_acc: 0.7145
Epoch 199/200
469/469 [==============================] - 3s - loss: 0.8342 - acc: 0.6911 - val_loss: 0.8607 - val_acc: 0.6709
Epoch 200/200
469/469 [==============================] - 3s - loss: 0.8421 - acc: 0.6907 - val_loss: 0.8015 - val_acc: 0.7150
